# Split final dataset into train, validation and test set

**Motivation:** Having prepared dataframes with respose and all predictores (version 1 and 2). This script aims to plit it into three distinct dataframes - train, validation, test

In [1]:
path = r''
file_name = r'analyze_split_statistics_dynasty_categorical.csv'

### Loads

In [2]:
import os
import sys
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import sklearn

from sklearn.model_selection import StratifiedShuffleSplit

# local
import set_path
import supp.support_load as sl
from supp.support_save import save_df

In [3]:
# print version of used packages
print("scikit-learn:", sklearn.__version__)
print("numpy:", np.__version__)
print("python:", sys.version)

scikit-learn: 1.4.2
numpy: 1.26.4
python: 3.11.8 | packaged by Anaconda, Inc. | (main, Feb 26 2024, 21:34:05) [MSC v.1916 64 bit (AMD64)]


In [4]:
# load database
dfs, dfs_name, dfs_export_date = sl.load_pickle()
iton, ntoi = sl.get_name_dicts(dfs_name)
dfs_export_date

Pickle database loaded.
C:\Users\Stoja\OneDrive\Documents\diplomka\scr\data\dfs_complete.pickle
Applied preprocessing: merge_on_jones
Applied preprocessing: remove_jones_duplicates


'2024-10-11'

In [5]:
# load feature version 1
df_v1 = sl.read_csv('df_vizier_titles_v1')
print(df_v1.shape)
df_v1.head(2)

CSV file loaded.
C:\Users\Stoja\OneDrive\Documents\diplomka\scr\data\df_vizier_titles_v1.csv
(3930, 46)


,ID_person,vizier,[imAxw xr] nTr aA,HAty-a,imAxw xr Wcir,imy-rA wabty,iry-pat,r P nb,mniw Nxn / zAw Nxn,imy-rA izwy (n) Xkr(w) ncwt,...,Xry-Hbt Hry-tp,xrp iAwt nbwt nTrwt,zS mDAt-nTr,Hry-tp Nxb,Htc(?) Inpw,imy iz Nxn,Hry-cStA n pr-dwAt,mdw rxyt,zA ncwt,imy-rA 5maw
0,322,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,323,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# load feature version 2
df_v2 = sl.read_csv('df_vizier_titles_v2_dynasty_categorical')
print(df_v2.shape)
df_v2.head(2)

CSV file loaded.
C:\Users\Stoja\OneDrive\Documents\diplomka\scr\data\df_vizier_titles_v2_dynasty_categorical.csv
(3841, 58)


,ID_person,vizier,mniw Nxn / zAw Nxn,imy iz Nxn,Hry-cStA,HAty-a,Xry-Hbt,cm / ctm,iwn knmwt,zA ncwt n Xt.f cmcw,...,imAxw xr Wcir,mDH ncwt qd(w) m prwy,aD-mr (n) zAb,father_was_vizier,dynasty_0,dynasty_1,dynasty_2,PC1,PC2,PC3
0,322,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,1.0,0.0,0.0,-0.775583,-0.261590,-0.035094
1,323,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,1.0,0.0,-0.375136,-0.465387,0.958193


In [7]:
# get common ids
person_ids_v1 = df_v1['ID_person'].to_list()
person_ids_v2 = df_v2['ID_person'].to_list()
common_ids = list(set(person_ids_v1).intersection(set(person_ids_v2)))
print(f'{len(common_ids)}\tcount of common_ids')

3841	count of common_ids


In [8]:
# get common features
features_v1 = df_v1.columns.to_list()
features_v2 = df_v2.columns.to_list()
common_features = list(set(features_v1).intersection(set(features_v2)))
all_features = list(set(features_v1).union(set(features_v2)))
print(f'{len(common_features)}\tcount of common_features')
print(f'{len(all_features)}\tcount of all_features')

46	count of common_features
58	count of all_features


In [9]:
# make one common dataframe
df_all = pd.merge(df_v1, df_v2, on=common_features, how='outer')
print(f'{df_all.shape}\tshape after merge')
df_all = df_all.loc[df_all['ID_person'].isin(common_ids), :]
print(f'{df_all.shape}\tshape after restricting to same persons')
df_all= df_all.set_index('ID_person')
print(f'{df_all.shape}\tshape after set_index to ID_person')
df_all.head(2)

(3930, 58)	shape after merge
(3841, 58)	shape after restricting to same persons
(3841, 57)	shape after set_index to ID_person


,vizier,[imAxw xr] nTr aA,HAty-a,imAxw xr Wcir,imy-rA wabty,iry-pat,r P nb,mniw Nxn / zAw Nxn,imy-rA izwy (n) Xkr(w) ncwt,wr 5 (m) pr 9Hwty,...,xrp (i)m(yw) nTrw,mDH zS(w) ncwt,mDH ncwt qd(w) m prwy,father_was_vizier,dynasty_0,dynasty_1,dynasty_2,PC1,PC2,PC3
ID_person,,,,,,,,,,,,,,,,,,,,,
322,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.775583,-0.261590,-0.035094
323,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.375136,-0.465387,0.958193


#### missings in version 1

In [10]:
# missings in version 1
missing_in_v1 = list(set(person_ids_v2) - set(person_ids_v1))
df_missing_in_v1 = df_v2[df_v2['ID_person'].isin(missing_in_v1)]
print(f'{len(missing_in_v1)}\tcount of missing_in_v1')
print(missing_in_v1)
df_missing_in_v1

0	count of missing_in_v1
[]


,ID_person,vizier,mniw Nxn / zAw Nxn,imy iz Nxn,Hry-cStA,HAty-a,Xry-Hbt,cm / ctm,iwn knmwt,zA ncwt n Xt.f cmcw,...,imAxw xr Wcir,mDH ncwt qd(w) m prwy,aD-mr (n) zAb,father_was_vizier,dynasty_0,dynasty_1,dynasty_2,PC1,PC2,PC3


#### missings in version 2

In [11]:
# missings in version 2
missing_in_v2 = list(set(person_ids_v1) - set(person_ids_v2))
df_missing_in_v2 = df_v1[df_v1['ID_person'].isin(missing_in_v2)]
print(f'{len(missing_in_v2)}\tcount of missing_in_v2')
print(missing_in_v2)
df_missing_in_v2

89	count of missing_in_v2
[519, 4619, 4621, 4622, 4623, 4632, 4634, 4635, 4640, 4147, 4162, 4704, 4197, 4729, 4730, 4731, 4732, 4223, 4746, 4748, 4749, 4750, 4751, 4752, 4754, 4763, 685, 4281, 212, 213, 1754, 1755, 1756, 1757, 1759, 3308, 4844, 3310, 3311, 3314, 263, 4365, 3855, 4884, 4885, 4375, 4376, 4377, 4887, 4380, 4382, 799, 4383, 4384, 4389, 4391, 4393, 4394, 4395, 4920, 4924, 4925, 4926, 4927, 3906, 4959, 4468, 3960, 4491, 4492, 4493, 4516, 2991, 3504, 3505, 3506, 3507, 3508, 3509, 3510, 4539, 4543, 4544, 4545, 4546, 4547, 3030, 4571, 4066]


,ID_person,vizier,[imAxw xr] nTr aA,HAty-a,imAxw xr Wcir,imy-rA wabty,iry-pat,r P nb,mniw Nxn / zAw Nxn,imy-rA izwy (n) Xkr(w) ncwt,...,Xry-Hbt Hry-tp,xrp iAwt nbwt nTrwt,zS mDAt-nTr,Hry-tp Nxb,Htc(?) Inpw,imy iz Nxn,Hry-cStA n pr-dwAt,mdw rxyt,zA ncwt,imy-rA 5maw
212,519,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
391,685,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
511,799,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1354,1754,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1355,1755,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3899,4924,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3900,4925,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3901,4926,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3902,4927,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# overview of omitted persons
df_missing_in_v2_overview = df_missing_in_v2.copy()
df_missing_in_v2_overview = df_missing_in_v2_overview.set_index('ID_person')
df_missing_in_v2_overview['total_titles_count'] = df_missing_in_v2_overview.sum(axis=1)
df_missing_in_v2_overview[['total_titles_count']].sort_values('total_titles_count', ascending=False)[:20]

,total_titles_count
ID_person,
4623,6.0
4920,5.0
4516,4.0
4493,4.0
3504,3.0
4632,3.0
4640,2.0
4162,2.0
4730,2.0


**Conclusion:** Omitted perosons had max 6 titles -> its low number

### define functions

In [13]:
def make_split(data, random_state):
    # Set response
    response = 'vizier'
    features_all = [col for col in data.columns if col != response]
    
    # Separating features and target
    X = data[features_all].values
    y = data[response].values
    
    # Stratified splitting for train, validation, and test sets
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=random_state)
    train_index, temp_index = next(sss.split(X, y))
    X_train, y_train = X[train_index], y[train_index]
    X_temp, y_temp = X[temp_index], y[temp_index]
    
    sss_temp = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
    val_index, test_index = next(sss_temp.split(X_temp, y_temp))
    
    X_val, y_val = X_temp[val_index], y_temp[val_index]
    X_test, y_test = X_temp[test_index], y_temp[test_index]

    # Split pandas dataframe
    data_train = data.iloc[train_index, :]
    data_val = data.iloc[temp_index, :].iloc[val_index, :]
    data_test = data.iloc[temp_index, :].iloc[test_index, :]

    return data_train, data_val, data_test

In [14]:
def get_split_stats(data, data_train, data_val, data_test):
    columns = data.columns.to_list()
    df_stat = pd.DataFrame(index=df_all.columns)
    # get counts
    df_stat['count_all'] = data.sum(axis=0)
    df_stat['count_train'] = data_train.sum(axis=0)
    df_stat['count_val'] = data_val.sum(axis=0)
    df_stat['count_test'] = data_test.sum(axis=0)
    # get means
    df_stat['mean_all'] = data.mean(axis=0)
    df_stat['mean_train'] = data_train.mean(axis=0)
    df_stat['mean_val'] = data_val.mean(axis=0)
    df_stat['mean_test'] = data_test.mean(axis=0)
    # get mean ratios
    df_stat['ratio_train'] = df_stat['mean_train'] / df_stat['mean_all']
    df_stat['ratio_val'] = df_stat['mean_val'] / df_stat['mean_all']
    df_stat['ratio_test'] = df_stat['mean_test'] / df_stat['mean_all']
    # get std
    df_stat['std_all'] = data.std(axis=0)
    df_stat['std_train'] = data_train.std(axis=0)
    df_stat['std_val'] = data_val.std(axis=0)
    df_stat['std_test'] = data_test.std(axis=0)
    # get std ratios
    df_stat['ratio_std_train'] = df_stat['std_train'] / df_stat['std_all']
    df_stat['ratio_std_val'] = df_stat['std_val'] / df_stat['std_all']
    df_stat['ratio_std_test'] = df_stat['std_test'] / df_stat['std_all']

    return df_stat

In [15]:
def analyze_split(df_stat, threshold):
    columns_ratio = ['ratio_train', 'ratio_val', 'ratio_test']
    continuous_feature = ['dyn_num', 'PC1', 'PC2', 'PC3']
    
    # binary features
    df_help = df_stat.loc[~df_stat.index.isin(continuous_feature), columns_ratio]
    mask = (df_help-1).abs()>threshold
    count_issue = mask.sum(axis=0)
    max_vals = df_help.max(axis=0)
    min_vals = df_help.min(axis=0)
    # continuous feature
    df_help = df_stat.loc[df_stat.index.isin(continuous_feature), columns_ratio]
    mask = (df_help-1).abs()>threshold
    count_issue_con = mask.sum(axis=0)
    max_vals_con = df_help.max(axis=0)
    min_vals_con = df_help.min(axis=0)

    # rename binary features
    count_issue.index = ['cnt_issue_train', 'cnt_issue_val', 'cnt_issue_test']
    max_vals.index = ['max_train', 'max_val', 'max_test']
    min_vals.index = ['min_train', 'min_val', 'min_test']

    # rename continuous feature
    count_issue_con.index = ['cnt_issue_train_con', 'cnt_issue_val_con', 'cnt_issue_test_con']
    max_vals_con.index = ['max_train_con', 'max_val_con', 'max_test_con']
    min_vals_con.index = ['min_train_con', 'min_val_con', 'min_test_con']

    # concat each stats
    result = pd.concat([count_issue, max_vals, min_vals,
                        count_issue_con, max_vals_con, min_vals_con], ignore_index=False)

    return result

In [16]:
def save_split_stat(stats, random_state, threshold):
    # File path
    file_path = os.path.join(path, file_name)
    
    # Convert Series to a one-row DataFrame
    rs = pd.Series({'random_state': random_state, 'threshold': threshold})
    s = pd.concat([rs, stats], ignore_index=False)
    # Prepend
    df_row = pd.DataFrame([s])
    
    # Check if file exists
    if not os.path.isfile(file_path):
        # File doesn't exist: create it with header
        df_row.to_csv(file_path, index=False)
    else:
        # File exists: append without header
        df_row.to_csv(file_path, mode='a', header=False, index=False)

### make multiple splits

In [17]:
# threshold = 0.5
# random_state_start = pd.read_csv(os.path.join(path, file_name))['random_state'].astype(int).max() + 1

In [18]:
# for random_state in range(random_state_start, 4294967295):
#     df_train, df_val, df_test = make_split(df_all, random_state)
#     df_stat = get_split_stats(df_all, df_train, df_val, df_test)
#     stats = analyze_split(df_stat, threshold)
#     save_split_stat(stats, random_state, threshold)
#     if random_state % 100 == 0:
#         print(f"random_state {random_state} finished")

### analyze splits

In [19]:
df_splits = pd.read_csv(os.path.join(path, file_name))
print(df_splits.shape)
df_splits.head(2)

(5817706, 20)


,random_state,threshold,cnt_issue_train,cnt_issue_val,cnt_issue_test,max_train,max_val,max_test,min_train,min_val,min_test,cnt_issue_train_con,cnt_issue_val_con,cnt_issue_test_con,max_train_con,max_val_con,max_test_con,min_train_con,min_val_con,min_test_con
0,0.0,0.5,0.0,2.0,13.0,1.180432,1.795339,2.048260,0.779424,0.0,0.289428,2.0,2.0,3.0,2.840667,0.582193,27.789058,-3.877286,-3.074900,-3.564249
1,1.0,0.5,0.0,11.0,12.0,1.326876,2.424874,1.901956,0.612404,0.0,0.332842,2.0,3.0,3.0,1.431784,1.514016,20.602400,-2.126807,-4.044668,-0.424062


In [20]:
threshold_lower = 0.5
threshold_upper = 2.0
mask1 = df_splits['min_train'] > threshold_lower
mask2 = df_splits['min_val'] > threshold_lower
mask3 = df_splits['min_test'] > threshold_lower
mask4 = df_splits['max_train'] < threshold_upper
mask5 = df_splits['max_val'] < threshold_upper
mask6 = df_splits['max_test'] < threshold_upper
mask7 = df_splits['cnt_issue_val'] < 4
mask8 = df_splits['cnt_issue_test'] < 4
mask = mask1 & mask2 & mask3 & mask4 & mask5 & mask6 & mask7 & mask8
df = df_splits.loc[mask, :]
print(df.shape)
df

(7, 20)


,random_state,threshold,cnt_issue_train,cnt_issue_val,cnt_issue_test,max_train,max_val,max_test,min_train,min_val,min_test,cnt_issue_train_con,cnt_issue_val_con,cnt_issue_test_con,max_train_con,max_val_con,max_test_con,min_train_con,min_val_con,min_test_con
1039759,1039759.0,0.5,0.0,3.0,1.0,1.122741,1.905258,1.566317,0.743051,0.740934,0.525540,3.0,3.0,3.0,3.513997,3.282133,7.910576,-0.771592,0.262323,-9.975256
1511290,1511290.0,0.5,0.0,2.0,3.0,1.099187,1.778241,1.997054,0.714472,0.512954,0.587369,2.0,3.0,3.0,2.593335,10.832748,9.646379,-2.647626,-10.325384,4.883080
1729859,1729859.0,0.5,0.0,1.0,2.0,1.180432,1.579359,1.901956,0.816539,0.555700,0.512065,3.0,2.0,3.0,2.537807,1.820732,-2.230349,1.900419,-2.434406,-6.500116
2447136,2447136.0,0.5,0.0,3.0,3.0,1.209106,1.961295,1.901956,0.714472,0.512954,0.512065,3.0,3.0,3.0,2.117664,7.066459,12.919001,-2.858461,-6.107771,-1.150199
3830838,3830838.0,0.5,0.0,0.0,3.0,1.143155,1.449653,1.849124,0.779424,0.512954,0.525540,2.0,2.0,3.0,1.717055,1.839024,15.274407,-0.001343,-14.686262,-3.178028
4174558,4174558.0,0.5,0.0,1.0,2.0,1.154147,1.905258,1.957896,0.756499,0.512954,0.640081,3.0,2.0,3.0,2.448109,1.252146,0.034647,2.063730,-2.997047,-5.997837
4878220,4878220.0,0.5,0.0,3.0,3.0,1.143155,1.818655,1.901956,0.816539,0.635086,0.605168,3.0,3.0,3.0,2.103438,11.445665,4.948197,-2.024792,0.013922,-3.156085


### make final split

In [21]:
# define number of the best split
threshold = 0.5
random_state = 4174558

In [22]:
# perform split according to the best random_state
df_train, df_val, df_test = make_split(df_all, random_state)
df_stat = get_split_stats(df_all, df_train, df_val, df_test)
stats = analyze_split(df_stat, threshold)
pd.DataFrame(stats).transpose()

,cnt_issue_train,cnt_issue_val,cnt_issue_test,max_train,max_val,max_test,min_train,min_val,min_test,cnt_issue_train_con,cnt_issue_val_con,cnt_issue_test_con,max_train_con,max_val_con,max_test_con,min_train_con,min_val_con,min_test_con
0,0.0,1.0,2.0,1.154147,1.905258,1.957896,0.756499,0.512954,0.640081,3.0,2.0,3.0,2.448109,1.252146,0.034647,2.06373,-2.997047,-5.997837


In [23]:
# check slits dimensions
print(f'DIMENSION CHECK')
print(f'{df_train.shape}\tshape of df_train')
print(f'{df_val.shape}\tshape of df_val')
print(f'{df_test.shape}\tshape of df_test')

# check slits dimensions
print(f'\nVIZIER COUNT CHECK')
print(f'{df_train["vizier"].sum()}\tcount of viziers in df_train')
print(f'{df_val["vizier"].sum()}\tcount of viziers in df_val')
print(f'{df_test["vizier"].sum()}\tcount of viziers in df_test')

# check slits dimensions
print(f'\nDISTICT ID_person CHECK')
id_train = set(df_train.index.to_list())
id_val = set(df_val.index.to_list())
id_test = set(df_test.index.to_list())
print(f'{id_train.intersection(id_val)}\tintersection of id_train and id_val')
print(f'{id_train.intersection(id_test)}\tintersection of id_train and id_test')
print(f'{id_val.intersection(id_test)}\tintersection of id_val and id_test')

DIMENSION CHECK
(2688, 57)	shape of df_train
(576, 57)	shape of df_val
(577, 57)	shape of df_test

VIZIER COUNT CHECK
56	count of viziers in df_train
12	count of viziers in df_val
12	count of viziers in df_test

DISTICT ID_person CHECK
set()	intersection of id_train and id_val
set()	intersection of id_train and id_test
set()	intersection of id_val and id_test


In [24]:
# print problematic features
threshold_low = 0.8
threshold_up =  1.4
columns = ['count_all', 'count_train', 'count_val', 'count_test', 'ratio_train', 'ratio_val', 'ratio_test',
           'mean_train', 'mean_val', 'mean_test']
mask_1 = (df_stat['ratio_train']<threshold_low) | (df_stat['ratio_train']>threshold_up)
mask_2 = (df_stat['ratio_val']<threshold_low)   | (df_stat['ratio_val']>threshold_up)
mask_3 = (df_stat['ratio_test']<threshold_low)  | (df_stat['ratio_test']>threshold_up)
mask = mask_1 | mask_2 | mask_3
df_stat.loc[mask, columns]

,count_all,count_train,count_val,count_test,ratio_train,ratio_val,ratio_test,mean_train,mean_val,mean_test
imAxw xr Wcir,72.000000,52.000000,13.000000,7.000000,1.032015,1.204017,0.647193,0.019345,0.022569,0.012132
mniw Nxn / zAw Nxn,30.000000,19.000000,4.000000,7.000000,0.904998,0.889120,1.553264,0.007068,0.006944,0.012132
imy-rA izwy (n) Xkr(w) ncwt,21.000000,12.000000,6.000000,3.000000,0.816539,1.905258,0.950978,0.004464,0.010417,0.005199
wr 5 (m) pr 9Hwty,17.000000,9.000000,3.000000,5.000000,0.756499,1.176777,1.957896,0.003348,0.005208,0.008666
Xry-tp ncwt,148.000000,114.000000,17.000000,17.000000,1.100673,0.765965,0.764638,0.042411,0.029514,0.029463
imy-rA prwy-nbw,20.000000,16.000000,2.000000,2.000000,1.143155,0.666840,0.665685,0.005952,0.003472,0.003466
iwn knmwt,61.000000,46.000000,5.000000,10.000000,1.077564,0.546590,1.091286,0.017113,0.008681,0.017331
zA ncwt n Xt.f,31.000000,22.000000,3.000000,6.000000,1.014089,0.645329,1.288422,0.008185,0.005208,0.010399
imy-rA kAt nbt (nt) ncwt,69.000000,53.000000,8.000000,8.000000,1.097594,0.773148,0.771808,0.019717,0.013889,0.013865
xrp SnDt nbt,26.000000,21.000000,2.000000,3.000000,1.154147,0.512954,0.768098,0.007812,0.003472,0.005199


In [25]:
# print problematic features
threshold_low = 0.8
threshold_up =  1.4
columns = ['mean_train', 'mean_val', 'mean_test', 'std_train', 'std_val', 'std_test']
df_stat.loc[['PC1', 'PC2', 'PC3'], columns]

,mean_train,mean_val,mean_test,std_train,std_val,std_test
PC1,0.010075,-0.007606,-0.010732,2.130061,2.475030,2.085683
PC2,-0.010243,0.014875,-0.000172,0.894410,1.005922,0.984550
PC3,-0.009998,-0.005114,0.024495,0.821549,0.791311,0.707576


### data postprocessing

#### titles with high correlation with vizier title

In [26]:
non_title_features = [
 'vizier',
 #'father_was_vizier',
 #'dynasty_0',
 #'dynasty_1',
 #'dynasty_2',
 'PC1',
 'PC2',
 'PC3'
]
title_all = [col for col in df_all.columns if col not in non_title_features]

In [27]:
df_all_stat = pd.DataFrame([], index=title_all)
df_all_stat['count'] = df_all.loc[:, title_all].sum(axis=0)
df_all_stat['count_vizier'] = df_all.loc[df_all['vizier']==1, title_all].sum(axis=0)
df_all_stat['count_non_vizier'] = df_all.loc[df_all['vizier']!=1, title_all].sum(axis=0)
df_all_stat['ratio'] = (df_all_stat['count_vizier'] / df_all_stat['count']).round(4)
df_all_stat = df_all_stat.sort_values('ratio', ascending=False)
df_all_stat.head()

,count,count_vizier,count_non_vizier,ratio
imy-rA Hwt-wrt 6,18.0,17.0,1.0,0.9444
wr 5 (m) pr 9Hwty,17.0,15.0,2.0,0.8824
xrp iAwt nbwt nTrwt,17.0,14.0,3.0,0.8235
aA 8wAw,20.0,16.0,4.0,0.8000
imy iz Nxn,14.0,11.0,3.0,0.7857


**NOTE:**
- "imy-rA Hwt-wrt 6" has higt correlation with vizier.
- It will be removed

In [28]:
title_name = 'imy-rA Hwt-wrt 6'
print(f'{df_all.shape}\tshape of df_all before removal')
df_all = df_all[[col for col in df_all.columns if col!=title_name]]
print(f'{df_all.shape}\tshape of df_all after removal')

(3841, 57)	shape of df_all before removal
(3841, 56)	shape of df_all after removal


In [29]:
title_name = 'PC3'
print(f'{df_all.shape}\tshape of df_all before removal')
df_all = df_all[[col for col in df_all.columns if col!=title_name]]
print(f'{df_all.shape}\tshape of df_all after removal')

(3841, 56)	shape of df_all before removal
(3841, 55)	shape of df_all after removal


#### problematic viziers

In [30]:
vizier__35_title_list = sl.read_csv('vizier__35_title_list')
vizier__35_title_list = vizier__35_title_list.set_index('ID_person')
mask_1 = (vizier__35_title_list['vizier']==1)
mask_2 = (vizier__35_title_list.sum(axis=1)<=2)
vizier_with_zero_titles = vizier__35_title_list.loc[mask_1 &  mask_2, :].index.to_list()
print(f'{vizier_with_zero_titles}\tviziers with zero titles')

CSV file loaded.
C:\Users\Stoja\OneDrive\Documents\diplomka\scr\data\unpacked\df_person_all\merged\vizier__35_title_list.csv
[481, 856, 2698, 3563, 3600]	viziers with zero titles


In [31]:
# these viziers have only vizier title, nothing else
df_help = vizier__35_title_list.loc[vizier_with_zero_titles, :]
df_help.loc[:, df_help.sum(axis=0)>0]

,vizier,tAyty TAty (n) zAb
ID_person,,
481,1,1.0
856,1,1.0
2698,1,1.0
3563,1,1.0
3600,1,1.0


**NOTE:**
- There are vizier, that have only vizier title, nothink else.
- There is no pattern to model on these persons, I will removed them.

In [32]:
print(f'{df_all.shape}\tshape of df_all before removal of viziers')
df_all = df_all.loc[~df_all.index.isin(vizier_with_zero_titles), :]
print(f'{df_all.shape}\tshape of df_all after removal of viziers')

(3841, 55)	shape of df_all before removal of viziers
(3836, 55)	shape of df_all after removal of viziers


### final analysis

In [33]:
print(f'ORIGINAL SHAPE')
print(f'{df_all.shape}\tshape of df_all')
print(f'{df_train.shape}\tshape of df_train')
print(f'{df_val.shape}\tshape of df_val')
print(f'{df_test.shape}\tshape of df_test')

columns_final = df_all.columns
df_train =  df_train.loc[df_train.index.isin(df_all.index), columns_final]
df_val = df_val.loc[df_val.index.isin(df_all.index), columns_final]
df_test = df_test.loc[df_test.index.isin(df_all.index), columns_final]

print(f'\nSHAPE AFTER POSTPROCESSING')
print(f'{df_all.shape}\tshape of df_all')
print(f'{df_train.shape}\tshape of df_train')
print(f'{df_val.shape}\tshape of df_val')
print(f'{df_test.shape}\tshape of df_test')

ORIGINAL SHAPE
(3836, 55)	shape of df_all
(2688, 57)	shape of df_train
(576, 57)	shape of df_val
(577, 57)	shape of df_test

SHAPE AFTER POSTPROCESSING
(3836, 55)	shape of df_all
(2684, 55)	shape of df_train
(575, 55)	shape of df_val
(577, 55)	shape of df_test


In [34]:
df_stat_new = get_split_stats(df_all, df_train, df_val, df_test)

In [35]:
# print problematic features
threshold_low = 0.8
threshold_up =  1.4
columns = ['count_all', 'count_train', 'count_val', 'count_test', 'ratio_train', 'ratio_val', 'ratio_test']
mask_1 = (df_stat_new['ratio_train']<threshold_low) | (df_stat_new['ratio_train']>threshold_up)
mask_2 = (df_stat_new['ratio_val']<threshold_low)   | (df_stat_new['ratio_val']>threshold_up)
mask_3 = (df_stat_new['ratio_test']<threshold_low)  | (df_stat_new['ratio_test']>threshold_up)
mask = mask_1 | mask_2 | mask_3
df_stat_new.loc[mask, columns]

,count_all,count_train,count_val,count_test,ratio_train,ratio_val,ratio_test
imAxw xr Wcir,72.000000,52.000000,13.000000,7.000000,1.032207,1.204541,0.646351
mniw Nxn / zAw Nxn,30.000000,19.000000,4.000000,7.000000,0.905166,0.889507,1.551242
imy-rA izwy (n) Xkr(w) ncwt,21.000000,12.000000,6.000000,3.000000,0.816692,1.906087,0.949740
wr 5 (m) pr 9Hwty,17.000000,9.000000,3.000000,5.000000,0.756641,1.177289,1.955347
Xry-tp ncwt,148.000000,114.000000,17.000000,17.000000,1.100878,0.766298,0.763642
imy-rA prwy-nbw,20.000000,16.000000,2.000000,2.000000,1.143368,0.667130,0.664818
iwn knmwt,61.000000,46.000000,5.000000,10.000000,1.077765,0.546828,1.089866
zA ncwt n Xt.f,31.000000,22.000000,3.000000,6.000000,1.014278,0.645610,1.286745
imy-rA kAt nbt (nt) ncwt,69.000000,53.000000,8.000000,8.000000,1.097799,0.773485,0.770804
xrp SnDt nbt,26.000000,21.000000,2.000000,3.000000,1.154362,0.513177,0.767098


In [36]:
# print problematic features
threshold_low = 0.8
threshold_up =  1.4
columns = ['mean_train', 'mean_val', 'mean_test', 'std_train', 'std_val', 'std_test']
df_stat.loc[['PC1', 'PC2', 'PC3'], columns]

,mean_train,mean_val,mean_test,std_train,std_val,std_test
PC1,0.010075,-0.007606,-0.010732,2.130061,2.475030,2.085683
PC2,-0.010243,0.014875,-0.000172,0.894410,1.005922,0.984550
PC3,-0.009998,-0.005114,0.024495,0.821549,0.791311,0.707576


### save

In [37]:
# define splits for each feature version
features_v1 = [col for col in df_v1.columns if col in df_all.columns]
df_train_v1 =  df_train.loc[:, features_v1]
df_val_v1 = df_val.loc[:, features_v1]
df_test_v1 = df_test.loc[:, features_v1]

features_v2 = [col for col in df_v2.columns if col in df_all.columns]
df_train_v2 =  df_train.loc[:, features_v2]
df_val_v2 = df_val.loc[:, features_v2]
df_test_v2 = df_test.loc[:, features_v2]

In [38]:
# check of version 1
print(f'VERSION 1, SHAPES')
print(f'{df_train_v1.shape}\tshape of df_train_v1')
print(f'{df_val_v1.shape}\tshape of df_val_v1')
print(f'{df_test_v1.shape}\tshape of df_test_v1')

print(f'\nVERSION 1, VIZIER COUNTS')
print(f'{df_train_v1["vizier"].sum()}\tcount of vizier in df_train_v1')
print(f'{df_val_v1["vizier"].sum()}\tcount of vizier in df_val_v1')
print(f'{df_test_v1["vizier"].sum()}\tcount of vizier in df_test_v1')

VERSION 1, SHAPES
(2684, 44)	shape of df_train_v1
(575, 44)	shape of df_val_v1
(577, 44)	shape of df_test_v1

VERSION 1, VIZIER COUNTS
52	count of vizier in df_train_v1
11	count of vizier in df_val_v1
12	count of vizier in df_test_v1


In [39]:
# check of version 2
print(f'VERSION 2, SHAPES')
print(f'{df_train_v2.shape}\tshape of df_train_v2')
print(f'{df_val_v2.shape}\tshape of df_val_v2')
print(f'{df_test_v2.shape}\tshape of df_test_v2')

print(f'\nVERSION 2, VIZIER COUNTS')
print(f'{df_train_v2["vizier"].sum()}\tcount of vizier in df_train_v2')
print(f'{df_val_v2["vizier"].sum()}\tcount of vizier in df_val_v2')
print(f'{df_test_v2["vizier"].sum()}\tcount of vizier in df_test_v2')

VERSION 2, SHAPES
(2684, 55)	shape of df_train_v2
(575, 55)	shape of df_val_v2
(577, 55)	shape of df_test_v2

VERSION 2, VIZIER COUNTS
52	count of vizier in df_train_v2
11	count of vizier in df_val_v2
12	count of vizier in df_test_v2


In [40]:
# SAVE
save_df(df_train_v1, 'df_vizier_train_v1_2', save_index=True)
save_df(df_val_v1, 'df_vizier_val_v1_2', save_index=True)
save_df(df_test_v1, 'df_vizier_test_v1_2', save_index=True)

save_df(df_train_v2, 'df_vizier_train_v2_2', save_index=True)
save_df(df_val_v2, 'df_vizier_val_v2_2', save_index=True)
save_df(df_test_v2, 'df_vizier_test_v2_2', save_index=True)

Dataframe saved into C:\Users\Stoja\OneDrive\Documents\diplomka\scr\data\df_vizier_train_v1_2.csv
Dataframe saved into C:\Users\Stoja\OneDrive\Documents\diplomka\scr\data\df_vizier_val_v1_2.csv
Dataframe saved into C:\Users\Stoja\OneDrive\Documents\diplomka\scr\data\df_vizier_test_v1_2.csv
Dataframe saved into C:\Users\Stoja\OneDrive\Documents\diplomka\scr\data\df_vizier_train_v2_2.csv
Dataframe saved into C:\Users\Stoja\OneDrive\Documents\diplomka\scr\data\df_vizier_val_v2_2.csv
Dataframe saved into C:\Users\Stoja\OneDrive\Documents\diplomka\scr\data\df_vizier_test_v2_2.csv


In [41]:
df_test_v2.columns

Index(['vizier', 'mniw Nxn / zAw Nxn', 'imy iz Nxn', 'Hry-cStA', 'HAty-a',
       'Xry-Hbt', 'cm / ctm', 'iwn knmwt', 'zA ncwt n Xt.f cmcw',
       'imy-rA gc-pr', 'iry-pat', 'aD-mr 8p', 'Htc(?) Inpw', 'wt(y) Inpw',
       'imy-rA Snwty', 'imy-rA 5maw', 'imAxw', 'imy-rA wabty',
       'imy-rA prwy-HD', 'imy-rA izwy (n) Xkr(w) ncwt', 'Xry-Hbt Hry-tp',
       'cmr waty', 'imy-rA prwy-nbw', 'r P nb', 'zS mDAt-nTr',
       'zA ncwt n Xt.f', 'Hry-tp Nxb', 'Hry-cStA n wD(t)-mdw nbt nt ncwt',
       'cmcw cnwt', 'zA ncwt', 'imy-rA kAt nbt (nt) ncwt', 'aA 8wAw', 'imA-a',
       'xrp (i)m(yw) nTrw', '[imAxw xr] nTr aA', 'xrp SnDt nbt', 'xrp aH',
       'wr 5 (m) pr 9Hwty', 'Hry-cStA n pr-dwAt', 'xrp iAwt nbwt nTrwt',
       'imy-rA zS(w) a(w) (nw) ncwt', 'Xry-tp ncwt', 'mDH zS(w) ncwt',
       'xtm(ty)-bity', 'imy iz', 'mdw rxyt', 'imAxw xr Wcir',
       'mDH ncwt qd(w) m prwy', 'aD-mr (n) zAb', 'father_was_vizier',
       'dynasty_0', 'dynasty_1', 'dynasty_2', 'PC1', 'PC2'],
      dtype='objec